# Imports

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import f1_score, roc_auc_score, classification_report, RocCurveDisplay
import matplotlib.pyplot as plt

This notebook mainly looks at RandomForest and Gradient Boosting as a contrast between bagging and boosting methods in ensemble learning.

Bagging: combine multiple models (RandomForest is a combination of multiple decision trees)
<br>
<br>
Boosting: sequential ensemble method that builds models one at a time, where each new model focuses on correcting the models of previous models.

# Read Data and Exploration

In [2]:
df = pd.read_csv('./data/train.csv')

X, y = df.drop(columns=['Survived']), df['Survived']

In [3]:
X.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Pclass       891 non-null    int64  
 2   Name         891 non-null    object 
 3   Sex          891 non-null    object 
 4   Age          714 non-null    float64
 5   SibSp        891 non-null    int64  
 6   Parch        891 non-null    int64  
 7   Ticket       891 non-null    object 
 8   Fare         891 non-null    float64
 9   Cabin        204 non-null    object 
 10  Embarked     889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 76.7+ KB


In [5]:
X.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

Looks like there are some NaN ages and cabin.

Simplifying the dataset a bit. The goal isn't to get the BEST predictor, but rather to play around with different preprocessing and scikit-learn's pipeline.

In [6]:
X = X[["Pclass", "Sex", "Age", "Fare", "Embarked"]]

# Train / Test Split

Separate 20% of the data and set aside for test set.

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Preprocessing Pipelines

### Numeric Pipeline

In [8]:
X_train.head()

,Pclass,Sex,Age,Fare,Embarked
692,3,male,NaN,56.4958,S
481,2,male,NaN,0.0000,S
527,1,male,NaN,221.7792,S
855,3,female,18.0,9.3500,S
801,2,female,31.0,26.2500,S


Age has missing values - need imputation.

Scale Fare and Pclass.

In [9]:
# Build pipline
numeric_pipeline = Pipeline([
    ('simple_imputer', SimpleImputer(strategy="median")),
    ('scaler', StandardScaler())
])

### Categorical Pipeline

In [10]:
categorical_pipeline = Pipeline([
    ('simple_imputer', SimpleImputer(strategy="most_frequent")),
    ('ohe', OneHotEncoder(handle_unknown="ignore"))
])

### Combine Numeric and Categorical Into One Preprocessor

In [11]:
preprocessor = ColumnTransformer(transformers=[
    ("Numeric", numeric_pipeline, ["Pclass","Age", "Fare"]),
    ("Categorical", categorical_pipeline, ["Sex", "Embarked"])
])

# Full Pipelines

In [12]:
rf_pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("model", RandomForestClassifier(random_state=42))
])

gb_pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("model", GradientBoostingClassifier(random_state=42))
])

# Fit Pipelines

### Hyperparam Tuning

In [ ]:
rf_param_grid = {
    "model__n_estimators": [100, 200, 500],
    "model__max_depth": [None, 5, 10],
    "model__min_samples_split": [2, 5]
}

gb_param_grid = {
    "model__n_estimators": [100, 200, 500],
    "model__learning_rate": [0.01, 0.1, 0.2],
    "model__max_depth": [3, 5]
}

print("Performing Grid Search for Random Forest")
rf_grid_search = GridSearchCV(
    rf_pipeline,
    rf_param_grid,
    cv=5,
    scoring="f1",
    n_jobs=-1,
    verbose=2
)

rf_grid_search.fit(X_train, y_train)

print(f"Best Random Forest parameters: {rf_grid_search.best_params_}")
print(f"Best Random Forest CV score: {rf_grid_search.best_score_:.4f}")

print("Performing Grid Search for Gradient Boosting")
gb_grid_search = GridSearchCV(
    gb_pipeline,
    gb_param_grid,
    cv=5,
    scoring="f1",
    n_jobs=-1,
    verbose=2
)

gb_grid_search.fit(X_train, y_train)

print(f"Best Gradient Boosting parameters: {gb_grid_search.best_params_}")
print(f"Best Gradient Boosting CV score: {gb_grid_search.best_score_:.4f}")

Performing Grid Search for Random Forest
Fitting 5 folds for each of 18 candidates, totalling 90 fits


[CV] END model__max_depth=None, model__min_samples_split=2, model__n_estimators=100; total time=   0.1s
[CV] END model__max_depth=None, model__min_samples_split=2, model__n_estimators=100; total time=   0.1s
[CV] END model__max_depth=None, model__min_samples_split=2, model__n_estimators=100; total time=   0.1s
[CV] END model__max_depth=None, model__min_samples_split=2, model__n_estimators=100; total time=   0.1s
[CV] END model__max_depth=None, model__min_samples_split=2, model__n_estimators=200; total time=   0.2s
[CV] END model__max_depth=None, model__min_samples_split=2, model__n_estimators=200; total time=   0.2s
[CV] END model__max_depth=None, model__min_samples_split=2, model__n_estimators=100; total time=   0.1s
[CV] END model__max_depth=None, model__min_samples_split=2, model__n_estimators=200; total time=   0.2s
[CV] END model__max_depth=None, model__min_samples_split=2, model__n_estimators=200; total time=   0.2s
[CV] END model__max_depth=None, model__min_samples_split=5, mode

# Evaluation

Best for random forest

In [ ]:
rf_model = rf_grid_search.best_estimator_
gb_model = gb_grid_search.best_estimator_

rf_grid_search.best_params_

{'model__max_depth': None,
 'model__min_samples_split': 5,
 'model__n_estimators': 100}

Best for gradient boosting

In [17]:
gb_grid_search.best_params_

{'model__learning_rate': 0.1,
 'model__max_depth': 5,
 'model__n_estimators': 100}

Evaluation on Test Set - Random Forest

In [ ]:
rf_test_preds = rf_model.predict(X_test)

print("Random Forest Test Set Results")
print("Random Forest F1 Score:", f1_score(y_test, rf_test_preds))
print("Random Forest ROC AUC:", roc_auc_score(y_test, rf_test_preds))

Random Forest Test Set Results
Random Forest F1 Score: 0.7596899224806202
Random Forest ROC AUC: 0.8050724637681159


Evaluation on Test Set - Gradient Boosting

In [23]:
gb_test_preds = gb_model.predict(X_test)

print("Gradient Boosting Test Set Results")
print("Gradient Boosting F1 Score:", f1_score(y_test, gb_test_preds))
print("Gradient Boosting ROC AUC:", roc_auc_score(y_test, gb_test_preds))



Gradient Boosting Test Set Results
Gradient Boosting F1 Score: 0.6991869918699187
Gradient Boosting ROC AUC: 0.7615942028985508


# Summary Table

In [25]:
summary_table = pd.DataFrame({
    "Model": ["Random Forest", "Gradient Boosting"],
    "Best Params": [rf_grid_search.best_params_, gb_grid_search.best_params_],
    "F1 Score": [f1_score(y_test, rf_test_preds), f1_score(y_test, gb_test_preds)],
    "ROC AUC": [roc_auc_score(y_test, rf_test_preds), roc_auc_score(y_test, gb_test_preds)]
})

summary_table

,Model,Best Params,F1 Score,ROC AUC
0,Random Forest,"{'model__max_depth': None, 'model__min_samples...",0.759690,0.805072
1,Gradient Boosting,"{'model__learning_rate': 0.1, 'model__max_dept...",0.699187,0.761594


Random Forest gave better F1 and ROC AUC. Let's go with this model.

Random Forest might have been better because this is a smaller dataset, and there is clean, interpretable features. There are also siople feature relationships and limited hyperparameteter tuning.

If this had been a larger dataset with more complex and noisy features, then gradient boosting might have been a more effective method. Gradient Boosting was a bit over kill.